<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/RAG_Retrieval_Optimization_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-cohere langchain pdfminer.six chromadb rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.4 MB/s eta 0:00:

In [ ]:
import os
from google.colab import userdata
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Hybrid Search

In [ ]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/hybrid-search"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)


In [ ]:
def load_data_to_vectordb(file_path,source):
  global bm25_retriever
  # Loop through a list of PDF files to process
  pages = []

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
              pages.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval
      if source == 1:
        bm25_retriever = BM25Retriever.from_documents(pages)
      else:
        db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/content/1706.03762v7.pdf",source=1)
load_data_to_vectordb(file_path="/content/1506.02640v5.pdf",source=2)

In [ ]:
# Initialize the BM25 retriever
bm25_retriever.k = 2  # Retrieve top 2 results
print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [ ]:
# Initialize retriever
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever_chromadb = docsearch.as_retriever(search_kwargs={"k": 5})

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_chromadb], weights=[0.3, 0.7]
)

In [ ]:
# Example query
query = "What is Self-Attention in Transformers?"

# Retrieve relevant documents/products
docs = ensemble_retriever.invoke(query)

docs

[Document(metadata={'retrived_from': 1, 'source': '/content/1506.02640v5.pdf'}, page_content='they are, and how they inter- act. The human visual system is fast and accurate, allow- ing us to perform complex tasks like driving with little con- scious thought. Fast, accurate algorithms for object detec- tion would allow computers to drive cars without special- ized sensors, enable assistive devices to convey real-time scene information to human users, and unlock the potential for general purpose, responsive robotic systems.  Current detection systems repurpose classiﬁers to per- form detection. To detect an object, these systems take a classiﬁer for that object and evaluate it at various locations and scales in a test image. Systems like deformable parts models (DPM) use a sliding window approach where the classiﬁer is run at evenly spaced locations over the entire image [10].  Figure 1: The YOLO Detection System. Processing images with YOLO is simple and straightforward. Our system (1)

In [ ]:
#Extract and print only the page content from each document
import pandas as pd

retrieval_df = pd.DataFrame()

page_content = []
retrieval_source = []
pdf_source = []

for doc in docs:
    page_content.append(doc.page_content)
    retrieval_source.append(doc.metadata['retrived_from'])
    pdf_source.append(doc.metadata['source'])

retrieval_df['page_content'] = page_content
retrieval_df['retrieval_source'] = retrieval_source
retrieval_df['pdf_source'] = pdf_source

retrieval_df.head(10)

,page_content,retrieval_source,pdf_source
0,"they are, and how they inter- act. The human v...",1,/content/1506.02640v5.pdf
1,Search [35] generates potential bounding boxes...,1,/content/1506.02640v5.pdf
2,to examine the accuracy-performance tradeoffs ...,1,/content/1506.02640v5.pdf
3,"You Only Look Once: Uniﬁed, Real-Time Object D...",1,/content/1506.02640v5.pdf
4,"Vision and Pattern Recognition (CVPR), 2013 IE...",1,/content/1506.02640v5.pdf
5,identically. This consists of two linear trans...,2,/content/1706.03762v7.pdf
6,our model contains no recurrence and no convol...,2,/content/1706.03762v7.pdf


## Re Ranking

In [ ]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/re-rank"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)

In [ ]:
def load_data_to_vectordb(file_path,source):
  # Loop through a list of PDF files to process

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval

      db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/content/1706.03762v7.pdf",source=1)
load_data_to_vectordb(file_path="/content/1506.02640v5.pdf",source=2)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [ ]:
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
# Initialize an empty DataFrame with specified columns
non_rerank_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Perform similarity search using a preconfigured document search tool
# This retrieves the top 3 documents based on relevance to the query
res_docs = docsearch.similarity_search_with_relevance_scores("What is the architecture of transformers?", k=3)

# Loop through the retrieved documents and populate the DataFrame
for doc in res_docs:
    non_rerank_df = non_rerank_df._append(
        {
            'Text': doc[0].page_content,  # Extract the page content (text) from the document
            'source': doc[0].metadata['source'],  # Extract the source metadata
            'relevance_score': doc[1]  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
non_rerank_df.head(3)

<ipython-input-72-b5c3429ed1f2>:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  non_rerank_df = non_rerank_df._append(


,Text,source,relevance_score
0,mechanism instead of sequence- aligned recurre...,/content/1706.03762v7.pdf,0.190476
1,as state of the art approaches in sequence mod...,/content/1706.03762v7.pdf,0.182639
2,"of 40K sentences. 7 Conclusion In this work,...",/content/1706.03762v7.pdf,0.118249


In [ ]:
# Import and initialize the reranker for document compression
compressor = CohereRerank()
# CohereRerank is a model or tool designed to rerank documents based on their relevance.

# Create a ContextualCompressionRetriever for improved document retrieval
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,               # Use the reranker as the base compression mechanism
    base_retriever=docsearch.as_retriever()   # Use the existing document search tool as the base retriever
)
# The ContextualCompressionRetriever combines the base retriever's results with reranking
# to provide more contextually relevant and concise results.


In [ ]:
# Initialize an empty DataFrame with specified columns
source_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Retrieve compressed documents relevant to the query using the contextual compression retriever
compressed_docs = compression_retriever.get_relevant_documents("What is the architecture of Transformers?")

# Loop through the first 3 compressed documents and populate the DataFrame
for i in range(3):
    source_df = source_df._append(
        {
            'Text': compressed_docs[i].page_content,  # Extract the content of the document
            'source': compressed_docs[i].metadata['source'],  # Extract the source information
            'relevance_score': compressed_docs[i].metadata['relevance_score']  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
source_df.head(3)

<ipython-input-73-7feac9a38528>:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  source_df = source_df._append({'Text':compressed_docs[i].page_content,'source':compressed_docs[i].metadata['source'],


,Text,source,relevance_score
0,mechanism instead of sequence- aligned recurre...,/content/1706.03762v7.pdf,0.997556
1,as state of the art approaches in sequence mod...,/content/1706.03762v7.pdf,0.973595
2,global dependencies between input and output. ...,/content/1706.03762v7.pdf,0.947088


We can see the last result is different and the relevance scores are also different but they are according to the relevance model